In [54]:
pip install pyzbar opencv-python

Note: you may need to restart the kernel to use updated packages.


In [55]:
pip install paho-mqtt

Note: you may need to restart the kernel to use updated packages.


In [56]:
from pyzbar import pyzbar
import paho.mqtt.client as mqtt
import cv2

In [57]:
#Informacion imagen
class InfoImg:
    def __init__(self, tipo, datos):
        self.tipo = tipo
        self.datos = datos

In [58]:
def decode(image):
    # Decodes all barcodes from an image
    decoded_objects = pyzbar.decode(image)

    if not decoded_objects:
        print("No barcodes detected.")
        return None  # Retorna None si no hay códigos de barras

    # Procesar solo el primer código de barras detectado
    obj = decoded_objects[0]
    print("Detected barcode:", obj)
    image = draw_barcode(obj, image)

    # Decodificar el tipo y los datos
    tipo = obj.type
    datos = obj.data.decode("utf-8")  # Decodificar los datos a texto
    print("Type:", tipo)
    print("Data:", datos)
    print()

    # Retornar una instancia de InfoImg
    return InfoImg(tipo, datos)


def draw_barcode(decoded, image):
    # n_points = len(decoded.polygon)
    # for i in range(n_points):
    #     image = cv2.line(image, decoded.polygon[i], decoded.polygon[(i+1) % n_points], color=(0, 255, 0), thickness=5)
    # uncomment above and comment below if you want to draw a polygon and not a rectangle
    image = cv2.rectangle(image, (decoded.rect.left, decoded.rect.top), 
                            (decoded.rect.left + decoded.rect.width, decoded.rect.top + decoded.rect.height),
                            color=(0, 255, 0),
                            thickness=5)
    return image

def publicador(infoImg, numeroGrupo):
    # PUBLICADOR
    client = mqtt.Client()
    client.connect("test.mosquitto.org", 1883, 60)

    # Caso 1: Primero en recibir el código
    # Sumar +10 a los últimos y pasar al segundo
    if str(infoImg.datos)[0] == str(numeroGrupo):
        siguiente_grupo = str(infoImg.datos)[1]  # Obtener el siguiente carácter
        nuevo_dato = int(infoImg.datos) + 10
        client.publish("PATRONES2025/G" + siguiente_grupo, nuevo_dato)
        print(f"Enviando el mensaje {nuevo_dato} al grupo {siguiente_grupo}")
        return  # Termina aquí para el Caso 1

    # Caso 2: Intermedio
    # Espera a que le llegue una respuesta
    # Sumar +10 a los últimos y pasar al siguiente
    def on_message(client, userdata, message):
        print(f"Mensaje recibido: {message.payload.decode('utf-8')}")
        nuevo_dato = int(message.payload.decode("utf-8")) + 10

        # Buscar el siguiente grupo en los datos
        datos_str = str(infoImg.datos)
        try:
            index_actual = datos_str.index(str(numeroGrupo))
            siguiente_grupo = datos_str[index_actual + 1]  # Obtener el siguiente carácter
        except (ValueError, IndexError):
            print("No se encontró el siguiente grupo en los datos.")
            client.loop_stop()  # Detener el bucle MQTT
            return

        # Publicar al siguiente grupo
        client.publish("PATRONES2025/G" + siguiente_grupo, nuevo_dato)
        print(f"Publicado al grupo {siguiente_grupo}: {nuevo_dato}")

        # Detener el bucle MQTT después de enviar el mensaje
        client.loop_stop()

    client.on_message = on_message
    client.subscribe("PATRONES2025/G" + str(numeroGrupo))
    print(f"Esperando mensaje en el grupo PATRONES2025/G{numeroGrupo}...")
    client.loop_start()


In [59]:
if __name__ == "__main__":
    from glob import glob

    barcodes = glob("Caso5.png")

    for barcode_file in barcodes:
        # Cargar la imagen con OpenCV
        img = cv2.imread(barcode_file)

        # Decodificar el primer código de barras detectado
        decoded_info = decode(img)

        if decoded_info is not None:
            print(f"Tipo: {decoded_info.tipo}, Datos: {decoded_info.datos}")
            # Intentar convertir los datos a un número
            try:
                datos = int(decoded_info.datos)  # Convertir los datos a un número entero si es posible
                publicador(decoded_info, 3)
                publicador(decoded_info, 1)

            except ValueError:
                print("No contiene datos, terminado programa")
                pass  # Si no es un número, mantener los datos como texto     
        else:
            print("No se detectaron códigos de barras.")

    

Detected barcode: Decoded(data=b'312546789760', type='DATABAR_EXP', rect=Rect(left=17, top=8, width=650, height=111), polygon=[Point(x=17, y=9), Point(x=17, y=119), Point(x=262, y=119), Point(x=507, y=118), Point(x=667, y=116), Point(x=667, y=8)], quality=141, orientation='UP')
Type: DATABAR_EXP
Data: 312546789760

Tipo: DATABAR_EXP, Datos: 312546789760


/tmp/ipykernel_10968/2344913289.py:38: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Enviando el mensaje 312546789770 al grupo 1
Esperando mensaje en el grupo PATRONES2025/G1...
